In [147]:
import pandas as pd
import matplotlib.pyplot as plt

from pyspark.sql.functions import monotonically_increasing_id

In [148]:
TweetsDf.head()

Row(user='vazquezsue', mensaje='La Universidad de Murcia se enorgullece de su compromiso social, pero la falta de diversidad y representación en el cuerpo docente y estudiantil es una preocupación. #UMU #FaltaDeDiversidad', likes='413')

Algoritmo usando SparkMlib

In [149]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.classification import LogisticRegression
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [150]:
# Palabras clave para tweets positivos y negativos
palabras_clave_positivas = ["privilegio", "maravilloso", "increíble", "genial", "emocionado","calidad"
                           ,"diversidad","paradisíacas","crecimiento","amigable","ideal","acogedora","innovador","emprendimiento",
                           "evolución","innovadores","emprendedor","excelencia","enriquecedora","oasis","enriquecedora","placentera",
                           "brillante","belleza","tesoro","inspiran","joya","fascinado"]




palabras_clave_negativas = ["triste", "decepcionado", "odio", "mal", "falta","desfasados","incompetencia","carece",
                            "corrupción","indignado","pesadilla","deficiente","discriminación","competitivo","muere",
                            "desigualdad","tortura","desalentadora","monotonía"]

In [151]:
# Función para asignar valor 1 si el mensaje contiene palabras clave positivas, 0 si contiene palabras clave negativas, y -1 si no contiene ninguna palabra clave
def clasificar_mensaje(mensaje):
    for palabra in palabras_clave_positivas:
        if palabra in mensaje:
            return 1
    for palabra in palabras_clave_negativas:
        if palabra in mensaje:
            return 0
    return -1

In [152]:
# Crear una sesión de Spark
spark = SparkSession.builder.appName("ClasificadorMensajes").getOrCreate()


ruta_archivo = "file:///home/student/Downloads/tweetsMejorado-2.csv"
df = spark.read.csv(ruta_archivo, header=True, inferSchema=True)


In [153]:
udf_clasificar_mensaje = udf(clasificar_mensaje, IntegerType())


In [154]:
# Agregar la nueva columna de clasificación al DataFrame
df = df.withColumn("clasificacion", udf_clasificar_mensaje(df["mensaje"]))

# Mostrar el DataFrame resultante
df.show(5)

+-----------------+--------------------+-----+-------------+
|             user|             mensaje|likes|clasificacion|
+-----------------+--------------------+-----+-------------+
|       vazquezsue|La Universidad de...|  413|            1|
|    samanthasmith|En la Universidad...|  929|            0|
|alexanderreynolds|La Universidad de...|  932|            1|
|        connielee|En la Universitat...|  395|            0|
|   zacharyjackson|La Universidad de...|  119|            0|
+-----------------+--------------------+-----+-------------+
only showing top 5 rows



In [155]:
# Mostrar las primeras 5 filas del DataFrame, mostrando solo las columnas 'usuario', 'mensaje' y 'clasificacion'


In [156]:
df_pandas = df.toPandas()

# Ahora df_pandas es un DataFrame de pandas
print(df_pandas.head())

                user                                            mensaje  \
0         vazquezsue  La Universidad de Murcia se enorgullece de su ...   
1      samanthasmith  En la Universidad de Barcelona no solo encuent...   
2  alexanderreynolds  La Universidad de Santiago de Compostela es un...   
3          connielee  En la Universitat de Lleida no solo encuentro ...   
4     zacharyjackson  La Universidad de Jaén es una institución que ...   

   likes  clasificacion  
0    413              1  
1    929              0  
2    932              1  
3    395              0  
4    119              0  


In [157]:
primera_fila = df_pandas.iloc[555]['mensaje']
# Mostrar la primera fila
print(primera_fila)

La Universitat Pompeu Fabra es un tesoro académico. Sus campus llenos de historia, su compromiso con la sostenibilidad y el ambiente acogedor hacen que estudiar aquí sea una experiencia inolvidable. #UPF #UniversidadSostenible


In [158]:
primera_fila = df_pandas.iloc[555]['clasificacion']
# Mostrar la primera fila
print(primera_fila)

1


In [159]:
filas_con_clasificacion_negativa = df_pandas[df_pandas['clasificacion'] == -1]

# Mostrar las filas con clasificacion igual a -1
print(filas_con_clasificacion_negativa)#-->NO HAY SIN CLASIFICAR

Empty DataFrame
Columns: [user, mensaje, likes, clasificacion]
Index: []


In [160]:
# Crear una sesión de Spark
spark = SparkSession.builder.appName("Conversion").getOrCreate()
# Crear un DataFrame de PySpark a partir del DataFrame de pandas
df_spark = spark.createDataFrame(df_pandas)
# Ahora df_spark es un DataFrame de PySpark
df_spark.show()

+-----------------+--------------------+-----+-------------+
|             user|             mensaje|likes|clasificacion|
+-----------------+--------------------+-----+-------------+
|       vazquezsue|La Universidad de...|  413|            1|
|    samanthasmith|En la Universidad...|  929|            0|
|alexanderreynolds|La Universidad de...|  932|            1|
|        connielee|En la Universitat...|  395|            0|
|   zacharyjackson|La Universidad de...|  119|            0|
|         reedtony|En la Universidad...|  579|            0|
|      vcunningham|Estudiar en la Un...|  681|            1|
|            jyang|En la Universitat...|  988|            0|
|    warnerrichard|En la Universidad...|  861|            1|
|             jlee|La Universidad de...|   15|            0|
|         jeremy09|¡La Universidad d...|  209|            1|
|          paula26|La Universitat de...|  493|            1|
|         brettkim|En la Universidad...|  247|            1|
|       irasmussen|La Un

# Modelo

In [161]:
spark = SparkSession.builder.appName("MLPipeline").getOrCreate()
train_data, test_data = df_spark.randomSplit([0.8, 0.2], seed=42)

In [162]:
print("Contenido del conjunto de prueba (test_data):")
etiquetas_reales_df = test_data.select("mensaje", "clasificacion")
test_data = test_data.drop("clasificacion")
test_data.show()

Contenido del conjunto de prueba (test_data):
+-----------------+--------------------+-----+
|             user|             mensaje|likes|
+-----------------+--------------------+-----+
|alexanderreynolds|La Universidad de...|  932|
|         andrea37|La Universidad de...|  498|
|         andrew17|¡La Universidad d...|  240|
|            bluna|Estudiar en la Un...|  621|
|          brian53|La Universidad Au...|  171|
|    burgesssandra|En la Universidad...|  159|
|       cristina87|Estudiar en la Un...|  949|
|     debbienguyen|En la Universidad...|  758|
|     edwardslaura|En la Universitat...|  558|
|    emilywilliams|¡La Universidad d...|  896|
|    erikabartlett|La Universitat de...|  570|
|           eyoung|¡La Universidad d...|   15|
|        foleytara|La Universidad de...|  183|
|         ginacarr|La Universidad de...|  919|
|            hwood|La Universidad de...|   29|
|         jeanne36|Estudiar en la Un...|   45|
|     josegonzalez|La Universidad de...|  554|
|  matthewslin

In [163]:
# Stage 1: Tokenizer
tokenizer = Tokenizer(inputCol="mensaje", outputCol="palabras")

# Stage 2: HashingTF
hashingTF = HashingTF(inputCol="palabras", outputCol="features")

# Stage 3: Logistic Regression
logistic_regression = LogisticRegression(maxIter=10, regParam=0.01, labelCol="clasificacion")

# Crear el pipeline con las etapas
pipeline = Pipeline(stages=[tokenizer, hashingTF, logistic_regression])


In [164]:
# Define the parameter grid for hyperparameter tuning (if needed)
paramGrid = (ParamGridBuilder()
             .addGrid(logistic_regression.regParam, [0.01, 0.1, 1.0])
             .build())

# Define the binary classification evaluator
evaluator = BinaryClassificationEvaluator(labelCol="clasificacion")

In [165]:
# Create the cross-validator for hyperparameter tuning (optional)
cross_validator = CrossValidator(estimator=pipeline,
                                 estimatorParamMaps=paramGrid,
                                 evaluator=evaluator,
                                 numFolds=3)

In [166]:
# Fit the cross-validator to the training data (or fit the pipeline if not using hyperparameter tuning)
model = cross_validator.fit(train_data)

# Make predictions on the test data
predictions = model.transform(test_data)

In [167]:
predictions

user,mensaje,likes,palabras,features,rawPrediction,probability,prediction
alexanderreynolds,La Universidad de...,932,"[la, universidad,...","(262144,[12967,14...",[-5.9030537661207...,[0.00272365340741...,1.0
andrea37,La Universidad de...,498,"[la, universidad,...","(262144,[4631,148...",[5.38893790162706...,[0.99545393986948...,0.0
andrew17,¡La Universidad d...,240,"[¡la, universidad...","(262144,[4631,234...",[5.97759847628605...,[0.99747150287746...,0.0
bluna,Estudiar en la Un...,621,"[estudiar, en, la...","(262144,[7188,234...",[-4.9756731857293...,[0.00685653317589...,1.0
brian53,La Universidad Au...,171,"[la, universidad,...","(262144,[4631,148...",[5.62983845411117...,[0.99642368087027...,0.0
burgesssandra,En la Universidad...,159,"[en, la, universi...","(262144,[220,2348...",[5.13090010960264...,[0.99412350006882...,0.0
cristina87,Estudiar en la Un...,949,"[estudiar, en, la...","(262144,[7188,234...",[-4.9571106415240...,[0.00698409928982...,1.0
debbienguyen,En la Universidad...,758,"[en, la, universi...","(262144,[7188,103...",[5.31619152941392...,[0.99511258102045...,0.0
edwardslaura,En la Universitat...,558,"[en, la, universi...","(262144,[4523,310...",[5.70278057641178...,[0.99667442170514...,0.0
emilywilliams,¡La Universidad d...,896,"[¡la, universidad...","(262144,[16800,23...",[5.92860724524051...,[0.99734488124129...,0.0


In [169]:
predictions_df = predictions


In [170]:
merged_df = predictions_df.join(etiquetas_reales_df, "mensaje")

# Visualizar el DataFrame unido
print("DataFrame unido:")
merged_df

DataFrame unido:


mensaje,user,likes,palabras,features,rawPrediction,probability,prediction,clasificacion
En la Universidad...,burgesssandra,159,"[en, la, universi...","(262144,[220,2348...",[5.13090010960264...,[0.99412350006882...,0.0,0
Estudiar en la Un...,heather59,592,"[estudiar, en, la...","(262144,[484,7188...",[4.91574473935805...,[0.99272308492727...,0.0,0
La Universitat Po...,housemariah,494,"[la, universitat,...","(262144,[31013,31...",[-5.4859113193279...,[0.00412764806648...,1.0,1
La Universitat de...,rthomas,751,"[la, universitat,...","(262144,[31013,31...",[-5.6199451035771...,[0.00361174808498...,1.0,1
En la Universidad...,jdavis,850,"[en, la, universi...","(262144,[220,2348...",[4.98331235325603...,[0.99319529041464...,0.0,0
La Universitat de...,erikabartlett,570,"[la, universitat,...","(262144,[4631,148...",[5.33637310841912...,[0.99520976044793...,0.0,0
En la Universidad...,millerrebecca,597,"[en, la, universi...","(262144,[7188,304...",[-4.7894753438431...,[0.00824822074329...,1.0,1
En la Universidad...,crystal62,489,"[en, la, universi...","(262144,[220,2348...",[5.35698863655790...,[0.99530704409888...,0.0,0
La Universidad de...,alexanderreynolds,932,"[la, universidad,...","(262144,[12967,14...",[-5.9030537661207...,[0.00272365340741...,1.0,1
¡La Universidad d...,theresa12,681,"[¡la, universidad...","(262144,[23484,31...",[4.92269325462130...,[0.99277310926527...,0.0,0


##Medir rendimiento

In [171]:
evaluator_binary = BinaryClassificationEvaluator(labelCol="clasificacion")

# Calcular la precisión del modelo
accuracy = evaluator_binary.evaluate(merged_df, {evaluator_binary.metricName: "areaUnderROC"})
print("Precisión del modelo (Area Under ROC):", accuracy)

Precisión del modelo (Area Under ROC): 1.0


In [ ]:
evaluator_multiclass = MulticlassClassificationEvaluator(labelCol="clasificacion")

# Calcular el f1-score del modelo
f1_score = evaluator_multiclass.evaluate(merged_df, {evaluator_multiclass.metricName: "f1"})
print("F1-Score del modelo:", f1_score)


In [174]:
#model.save("ModeloEntrenado")
model.write().save("file:///home/student/Downloads/ModeloEntrenado_Parametros")

In [ ]:
model.load()